In [1]:
%matplotlib qt5
from process_files import *
from func_target import *

qt.qpa.xcb: X server does not support XInput 2


qt.qpa.xcb: QXcbConnection: XCB error: 1 (BadRequest), sequence: 169, resource id: 132, major code: 130 (Unknown), minor code: 47


In [2]:
iline = 0
line = 'Fe_I_6173'
# calib data
config = configobj.ConfigObj('config.ini')
dkdir = config['darks']['directory']
tgdir = config['targetplate']['directory']
settings = [f for f in os.listdir(tgdir) if 'settings' in f]
settings = tgdir + os.sep + settings[0]
settings = configobj.ConfigObj(settings)
#
linestr = 'Line_' + str(get_line_num(settings, line, iline))
nacc = int(settings[linestr]['Polarimeter\\NAccumulations'])
nwav = int(settings[linestr]['NWavePoints'])
filtstr = settings[linestr]['Filter']
modstr = settings[linestr]['Polarimeter\\Modulation']
nmod = int(settings[filtstr]['Polarimeter\\'+modstr+'\\NModulations'])
nfpc = nmod*nacc*nwav
nfpw = nmod*nacc
ncyc = len(os.listdir(tgdir+os.sep+line))//3

In [3]:
# dark frames
dk0 = pf.open(config['darks'][line+'/bbi'])[0].data
dk1 = pf.open(config['darks'][line+'/pol1'])[0].data
dk2 = pf.open(config['darks'][line+'/pol2'])[0].data
# flat frames
ff_bbi = pf.open(config['flats'][line+'/bbi'])[0].data
ff_pol = pf.open(config['flats'][line+'/pol'])[0].data

In [4]:
# Get target
tg0, tg1, tg2 = 0.0, 0.0, 0.0 
dc0 = data_cube(tgdir, line, 0, 0)
print('Reading BBI target image from ', dc0.file)
tg0 = np.average(dc0.data, axis=2)-dk0[:,:,0]
tg0 = tg0/ff_bbi
tg0 = (tg0-tg0.min())/(tg0.max()-tg0.min())
dc1 = data_cube(tgdir, line, 1, 0)
print('Reading POL1 target image from ', dc1.file)
tg1 = np.average(dc1.data, axis=2)-dk1[:,:,0]
tg1 = tg1/ff_pol[:,:,0,0]
tg1 = np.flipud(tg1)
tg1 = (tg1-tg1.min())/(tg1.max()-tg1.min())
dc2 = data_cube(tgdir, line, 2, 0)
print('Reading POL2 target image from ', dc2.file)
tg2 = np.average(dc2.data, axis=2)-dk2[:,:,0]
tg2 = tg2/ff_pol[:,:,4,0]
tg2 = (tg2-tg2.min())/(tg2.max()-tg2.min())
#
config['targetplate']['flip1_lr'] = 0
config['targetplate']['flip1_ud'] = 1
config['targetplate']['flip2_lr'] = 0
config['targetplate']['flip2_ud'] = 0
config.write()
#

Reading BBI target image from  /arch/1/VTT/hellride/raw/20230504/Targetplate_1203/Fe_I_6173/HELLRIDE_bbi_20230504_120328106_tp.DAT
Reading POL1 target image from  /arch/1/VTT/hellride/raw/20230504/Targetplate_1203/Fe_I_6173/HELLRIDE_pol1_20230504_120328391_tp.DAT
Reading POL2 target image from  /arch/1/VTT/hellride/raw/20230504/Targetplate_1203/Fe_I_6173/HELLRIDE_pol2_20230504_120328693_tp.DAT


In [10]:
# tg0 = np.uint8(255*tg0)
# tg1 = np.uint8(255*tg1)
# tg2 = np.uint8(255*tg2)
blink_frames([tg0, tg1, tg2])

In [11]:
npts=12
plt.imshow(tg0)
pts0 = plt.ginput(n=npts, timeout=0, mouse_add=None, mouse_pop=None, mouse_stop=None)
plt.imshow(tg1)
pts1 = plt.ginput(n=npts, timeout=0, mouse_add=None, mouse_pop=None, mouse_stop=None)
plt.imshow(tg2)
pts2 = plt.ginput(n=npts, timeout=0, mouse_add=None, mouse_pop=None, mouse_stop=None)


KeyboardInterrupt: 

In [ ]:
def compute_angle1(img):
    """ 
    |   Compute the angle of the lines in the image of a grid
    |   Input : 2d image array
    |   Output : angle [0,90)
    """
    ny, nx = img.shape
    nxx, nyy = 3*nx//4-nx//4, 3*ny//4-ny//4
    td = 1.0*img[ny//4:ny//4+nyy, nx//4:nx//4+nxx]
    # td_plane = fit_2d_plane(td, w=td)
    # td -= td_plane
    # td = (td-td.min())/(td.max()-td.min())
    td = sobel(td)
    # compute threshold to generate binary image
    # thresh = 0.9*compute_thresh(td)
    thresh = 0
    td = np.array(td<thresh, dtype=np.uint8)
    plt.figure()
    plt.imshow(td)
    # detect lines using Hough transform
    lines = cv2.HoughLinesP(td, rho=1, theta=np.pi/180, threshold=nxx//4, lines=50, minLineLength=nxx//1.5, maxLineGap=8)
    ls = 0.0*td
    for line in lines:
        for x1, y1, x2, y2 in line:
            plt.plot([x1,x2],[y1,y2],color='red')
    # compute slope of the lines
    angs = (lines[:,0,1]-lines[:,0,3])/(lines[:,0,0]-lines[:,0,2])
    angs = np.degrees(np.arctan(angs))
    angs[np.argwhere(angs<0)] += 90 
    ang = np.median(angs)
    # remove extremes
    try:
        extremes = np.argwhere(np.abs(angs-ang)>3).flatten()
        angs = list(angs)
        del angs[extremes]
        ang = np.median(np.array(angs))
    except: pass
    return ang

def get_imcenter(img, nxx, nyy):
    ny, nx = img.shape
    x1, y1 = (nx-nxx)//2, (ny-nyy)//2
    x2, y2 = x1+nxx, y1+nyy
    return img[y1:y2,x1:x2]

def compute_shift(img1, img2):
    ny, nx = np.array(img1.shape)//2
    im1_cen = get_imcenter(img1, nx, ny)
    im1_int = fit_2d_plane(im1_cen, w=im1_cen)
    im2_cen = get_imcenter(img2, nx, ny)
    im2_int = fit_2d_plane(im2_cen, w=im2_cen)
    im1, im2 = im1_cen-im1_int*0.0, im2_cen-im2_int*0.0
    im1, im2 = im1/im1.mean(), im2/im2.mean()
    # Window
    hamm_0 = np.reshape(np.hamming(im1.shape[0]), (im1.shape[0], 1))
    # hamm_0[32:-32] = hamm_0[31]
    hamm_0 /= hamm_0.max()
    hamm_1 = np.reshape(np.hamming(im1.shape[1]), (1, im1.shape[1]))
    # hamm_1[32:-32] = hamm_1[31]
    hamm_1 /= hamm_1.max()
    w = hamm_0*hamm_1
    # Images
    im1, im2 = w*im1, w*im2
    fim1, fim2 = np.fft.fft2(im1), np.fft.fft2(im2)
    corr = np.fft.ifftshift(np.abs(np.fft.ifft2(fim1*np.conj(fim2))))
    half = np.array([ny//2, nx//2])
    sh = np.argwhere(corr==corr.max()).flatten()[0:2]-half
    return sh

def fit_2d_plane(z, w=1):
    """
    |   Fit the 2d data with the equation of z = ax + by + c
    |   Input:  2d array
    |   Output: fitted 2d array
    """
    ny, nx = z.shape
    y, x = np.meshgrid(np.arange(ny), np.arange(nx))
    s_1 = np.sum(np.ones([ny,nx])*w)
    s_x = np.sum(x*w)
    s_y = np.sum(y*w)
    s_z = np.sum(z*w)
    s_xx = np.sum(x**2*w)
    s_yy = np.sum(y**2*w)
    s_xy = np.sum(x*y*w)
    s_zx = np.sum(z*x*w)
    s_zy = np.sum(z*y*w)
    M_z = np.matrix([[s_z],[s_zx],[s_zy]])
    M_xy = np.matrix([[s_x, s_y, s_1],[s_xx, s_xy, s_x],[s_xy, s_yy, s_y]])
    M_abc = np.linalg.pinv(M_xy)*M_z
    a, b, c = np.array(M_abc).flatten()
    zfit = a*x + b*y + c
    return zfit

In [ ]:
# Rotation
ang0 = compute_angle1(tg0)
ang1 = compute_angle1(tg1)
ang2 = compute_angle1(tg2)
print('Angles: ', ang0, ang1, ang2)
rotang1 = ang1 - ang0
rotang2 = ang2 - ang0
tg0_rot = tg0
tg1_rot = rotate(tg1, rotang1, reshape=False)
tg2_rot = rotate(tg2, rotang2, reshape=False)
print('Rotation angles are: ', rotang1, rotang2)
config['targetplate']['rotang1'] = rotang1
config['targetplate']['rotang2'] = rotang2
config.write()
#

In [ ]:
# Scaling
mag1 = compute_magnification(tg0_rot, tg1_rot, minmag=0.98)
mag2 = compute_magnification(tg0_rot, tg2_rot, maxmag=1.02)
mag = 0.5*(mag1+mag2)
mag = 1
print('Magnification: ', mag1, mag2)
tg1_mag = zoom(tg1_rot, mag)
tg2_mag = zoom(tg2_rot, mag)
hpad = tg1_mag.shape[1]-tg0.shape[1]
vpad = tg1_mag.shape[0]-tg0.shape[0]
lpad, rpad = int(hpad/2), hpad-int(hpad/2)
tpad, bpad = int(vpad/2), vpad-int(vpad/2)
tg0_mag = np.pad(tg0_rot, ((tpad,bpad),(lpad,rpad)))
config['targetplate'][line+'/mag'] = mag
config.write()

In [ ]:
# Translation
sh1 = compute_shift(tg0_mag, tg1_mag)
sh2 = compute_shift(tg0_mag, tg2_mag)
print('Shifts: ', sh1, sh2)
tg0_shift = tg0_mag
tg1_shift = shift(tg1_mag, sh1)
tg2_shift = shift(tg2_mag, sh2)
config['targetplate'][line+'/shift1_y'] = sh1[0]
config['targetplate'][line+'/shift1_x'] = sh1[1]
config['targetplate'][line+'/shift2_y'] = sh2[0]
config['targetplate'][line+'/shift2_x'] = sh2[1]
config.write()

In [ ]:
plt.figure(); plt.imshow(tg0_shift)
plt.figure(); plt.imshow(tg1_shift)
plt.figure(); plt.imshow(tg2_shift)